In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# --- 1. CẤU HÌNH & KHỞI TẠO ---
DATA_PATH = '/kaggle/input/MABe-mouse-behavior-detection/'
print("Đang đọc metadata...")
try:
    df_train_meta = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
except FileNotFoundError:
    print("LỖI: Không tìm thấy file train.csv. Hãy kiểm tra lại đường dẫn dataset!")

# --- 2. HÀM TẠO ĐẶC TRƯNG (FEATURE ENGINEERING) ---
def calculate_features_with_memory(df):
    # a. Tính Khoảng cách & Vận tốc cơ bản
    try:
        dx = df['mouse1_body_center_x'] - df['mouse2_body_center_x']
        dy = df['mouse1_body_center_y'] - df['mouse2_body_center_y']
        df['distance'] = np.sqrt(dx**2 + dy**2)
    except KeyError:
        df['distance'] = 0 # Trường hợp không có chuột số 2
        
    # Vận tốc (Mouse 1)
    vx = df['mouse1_body_center_x'].diff().fillna(0)
    vy = df['mouse1_body_center_y'].diff().fillna(0)
    df['velocity_m1'] = np.sqrt(vx**2 + vy**2)
    
    # Vận tốc (Mouse 2)
    try:
        vx2 = df['mouse2_body_center_x'].diff().fillna(0)
        vy2 = df['mouse2_body_center_y'].diff().fillna(0)
        df['velocity_m2'] = np.sqrt(vx2**2 + vy2**2)
    except KeyError:
        df['velocity_m2'] = 0
        
    # b. Tạo Ký ức (Rolling Window - 10 frames)
    w = 10
    df['dist_mean_10'] = df['distance'].rolling(window=w).mean().fillna(0)
    df['dist_std_10'] = df['distance'].rolling(window=w).std().fillna(0)
    df['vel1_mean_10'] = df['velocity_m1'].rolling(window=w).mean().fillna(0)
    df['vel2_mean_10'] = df['velocity_m2'].rolling(window=w).mean().fillna(0)
    
    return df

# --- 3. HÀM LOAD & CHUẨN HÓA DỮ LIỆU ---
def get_video_data_normalized(idx):
    # Lấy thông tin metadata
    row = df_train_meta.iloc[idx]
    lab_id = row['lab_id']
    video_id = row['video_id']
    
    # Lấy tỉ lệ quy đổi (Pixel per CM)
    pix_per_cm = row['pix_per_cm_approx']
    if pd.isna(pix_per_cm) or pix_per_cm == 0:
        pix_per_cm = 1.0 
    
    print(f"Loading Video {video_id} (Lab: {lab_id}) - Scale: {pix_per_cm} pix/cm")
    
    # Đường dẫn file
    t_path = os.path.join(DATA_PATH, 'train_tracking', lab_id, f'{video_id}.parquet')
    a_path = os.path.join(DATA_PATH, 'train_annotation', lab_id, f'{video_id}.parquet')
    
    try:
        df_track = pd.read_parquet(t_path)
    except FileNotFoundError:
        print(f"-> Không tìm thấy file: {t_path}")
        return None
        
    # Pivot (Xoay bảng dọc -> ngang)
    px = df_track.pivot(index='video_frame', columns=['mouse_id', 'bodypart'], values='x')
    px.columns = [f"mouse{m}_{bp}_x" for m, bp in px.columns]
    
    py = df_track.pivot(index='video_frame', columns=['mouse_id', 'bodypart'], values='y')
    py.columns = [f"mouse{m}_{bp}_y" for m, bp in py.columns]
    
    df_wide = pd.concat([px, py], axis=1).sort_index(axis=1)
    
    # CHUẨN HÓA: Chia cho pix_per_cm để ra đơn vị CM
    df_wide = df_wide / pix_per_cm
    
    # Load & Gán nhãn
    try:
        df_annot = pd.read_parquet(a_path)
        df_wide['label'] = 'other'
        for _, row in df_annot.iterrows():
            if row['stop_frame'] <= df_wide.index.max():
                df_wide.loc[row['start_frame']:row['stop_frame'], 'label'] = row['action']
    except:
        df_wide['label'] = 'unknown' # Nếu không có file nhãn

    return df_wide.fillna(0)

# --- 4. CHẠY QUY TRÌNH HUẤN LUYỆN (PIPELINE) ---

# A. Load dữ liệu (Train 1 video, Test 1 video khác)
print("\n--- BẮT ĐẦU LOAD DỮ LIỆU ---")
df_train = get_video_data_normalized(0) # Video đầu tiên
df_test = get_video_data_normalized(1)  # Video thứ hai

if df_train is not None and df_test is not None:
    # B. Feature Engineering
    print("-> Đang tạo đặc trưng (Memory + Physics)...")
    df_train = calculate_features_with_memory(df_train)
    df_test = calculate_features_with_memory(df_test)
    
    # C. Chọn các cột đặc trưng quan trọng
    features = [
        'distance', 'velocity_m1', 'velocity_m2', 
        'dist_mean_10', 'dist_std_10', 
        'vel1_mean_10', 'vel2_mean_10'
    ]
    
    X_train = df_train[features]
    y_train_raw = df_train['label']
    
    X_test = df_test[features]
    y_test_raw = df_test['label']
    
    # D. Mã hóa nhãn (Label Encoding)
    le = LabelEncoder()
    # Fit trên cả 2 tập để biết hết các loại hành vi
    le.fit(list(y_train_raw.unique()) + list(y_test_raw.unique()))
    
    y_train = le.transform(y_train_raw)
    y_test = le.transform(y_test_raw)
    
    # E. Huấn luyện Model (Balanced Weights)
    print("\n--- ĐANG HUẤN LUYỆN MODEL ---")
    model = RandomForestClassifier(
        n_estimators=50, 
        random_state=42, 
        n_jobs=-1,
        class_weight='balanced' # Trọng số cân bằng (Phạt nặng nếu đoán sai nhãn hiếm)
    )
    model.fit(X_train, y_train)
    
    # F. Đánh giá
    print("\n--- KẾT QUẢ ĐÁNH GIÁ ---")
    y_pred = model.predict(X_test)
    
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.2%}")
    print("\nBáo cáo chi tiết:")
    print(classification_report(y_test, y_pred, target_names=le.classes_, zero_division=0))
    
    # G. Xem độ quan trọng
    imps = pd.DataFrame({
        'Feature': features, 
        'Importance': model.feature_importances_
    }).sort_values(by='Importance', ascending=False)
    print("\nTop Features quan trọng nhất:")
    print(imps)

Đang đọc metadata...

--- BẮT ĐẦU LOAD DỮ LIỆU ---
Loading Video 44566106 (Lab: AdaptableSnail) - Scale: 16.0 pix/cm
Loading Video 143861384 (Lab: AdaptableSnail) - Scale: 9.7 pix/cm
-> Đang tạo đặc trưng (Memory + Physics)...

--- ĐANG HUẤN LUYỆN MODEL ---

--- KẾT QUẢ ĐÁNH GIÁ ---
Accuracy: 42.21%

Báo cáo chi tiết:
              precision    recall  f1-score   support

    approach       0.00      0.00      0.00         0
      attack       0.02      0.00      0.00      7241
       avoid       0.03      0.04      0.03      1846
       chase       0.03      0.00      0.00      4376
 chaseattack       0.00      0.00      0.00      1102
       other       0.85      0.50      0.63     75100
        rear       0.00      0.00      0.00         0
      submit       0.00      0.00      0.00         0

    accuracy                           0.42     89665
   macro avg       0.11      0.07      0.08     89665
weighted avg       0.71      0.42      0.53     89665


Top Features quan trọng nhất

In [2]:
# --- BƯỚC 12: UNDERSAMPLING (PHIÊN BẢN AN TOÀN TUYỆT ĐỐI) ---
from sklearn.utils import resample

# 1. Tách nhóm
# (Đảm bảo df_train_final đã được tạo từ các bước trước)
df_train_balanced = df_train.copy() # Lấy từ biến df_train (hoặc df_train_final) của bạn

df_others = df_train_balanced[df_train_balanced['label'] == 'other']
df_actions = df_train_balanced[df_train_balanced['label'] != 'other']

n_others = len(df_others)
n_actions = len(df_actions)

print(f"Thống kê trước khi cắt: Other = {n_others}, Action = {n_actions}")

# 2. Tìm mẫu số chung (Lấy theo phe ít hơn)
min_samples = min(n_others, n_actions)
print(f"-> Sẽ cân bằng cả 2 về số lượng: {min_samples}")

# 3. Cắt gọt cả 2 phe về mức min_samples
# (Phe nào vốn đã ít thì giữ nguyên, phe nào nhiều thì bị cắt bớt)
df_others_bal = resample(df_others, 
                         replace=False, 
                         n_samples=min_samples, 
                         random_state=42)

df_actions_bal = resample(df_actions, 
                          replace=False, 
                          n_samples=min_samples, 
                          random_state=42)

# 4. Ghép lại
df_train_new = pd.concat([df_others_bal, df_actions_bal])
print(f"Sau khi cắt: Tổng cộng = {len(df_train_new)} dòng (Tỷ lệ 50-50)")

# 5. Huấn luyện lại
X_train_bal = df_train_new[features] # Dùng list features cũ
y_train_bal = le.transform(df_train_new['label'])

print("\n--- ĐANG TRAIN MODEL TRÊN DỮ LIỆU 50:50 ---")
model_under = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
model_under.fit(X_train_bal, y_train_bal)

# 6. Dự đoán & Kết quả
print("Đang dự đoán...")
y_pred_under = model_under.predict(X_test) 

print(f"\n>>> ĐỘ CHÍNH XÁC (Undersampling): {accuracy_score(y_test, y_pred_under):.2%}")
print("\nChi tiết hiệu năng (Tìm Recall của Attack):")
print(classification_report(y_test, y_pred_under, target_names=le.classes_, zero_division=0))

Thống kê trước khi cắt: Other = 7258, Action = 11193
-> Sẽ cân bằng cả 2 về số lượng: 7258
Sau khi cắt: Tổng cộng = 14516 dòng (Tỷ lệ 50-50)

--- ĐANG TRAIN MODEL TRÊN DỮ LIỆU 50:50 ---
Đang dự đoán...

>>> ĐỘ CHÍNH XÁC (Undersampling): 45.34%

Chi tiết hiệu năng (Tìm Recall của Attack):
              precision    recall  f1-score   support

    approach       0.00      0.00      0.00         0
      attack       0.04      0.00      0.00      7241
       avoid       0.03      0.05      0.04      1846
       chase       0.05      0.00      0.00      4376
 chaseattack       0.00      0.00      0.00      1102
       other       0.84      0.54      0.66     75100
        rear       0.00      0.00      0.00         0
      submit       0.00      0.00      0.00         0

    accuracy                           0.45     89665
   macro avg       0.12      0.07      0.09     89665
weighted avg       0.71      0.45      0.55     89665



In [3]:
# --- BƯỚC 13: KHẮC PHỤC LỖI MULTI-AGENT (CHỈ LẤY NHÃN CỦA CẶP 1-2) ---

def get_video_data_filtered(idx):
    # 1. Load Metadata & Quy đổi Pixel (Như cũ)
    row = df_train_meta.iloc[idx]
    lab_id = row['lab_id']
    video_id = row['video_id']
    pix_per_cm = row['pix_per_cm_approx'] if row['pix_per_cm_approx'] > 0 else 1.0
    
    print(f"Video {video_id}: Đang xử lý cặp Mouse 1 & Mouse 2...")
    
    # 2. Load Tracking & Chuẩn hóa (Như cũ)
    t_path = os.path.join(DATA_PATH, 'train_tracking', lab_id, f'{video_id}.parquet')
    a_path = os.path.join(DATA_PATH, 'train_annotation', lab_id, f'{video_id}.parquet')
    
    try:
        df_track = pd.read_parquet(t_path)
    except FileNotFoundError: return None

    # Pivot
    px = df_track.pivot(index='video_frame', columns=['mouse_id', 'bodypart'], values='x')
    px.columns = [f"mouse{m}_{bp}_x" for m, bp in px.columns]
    py = df_track.pivot(index='video_frame', columns=['mouse_id', 'bodypart'], values='y')
    py.columns = [f"mouse{m}_{bp}_y" for m, bp in py.columns]
    df_wide = pd.concat([px, py], axis=1).sort_index(axis=1)
    df_wide = df_wide / pix_per_cm # Chuẩn hóa cm

    # 3. --- QUAN TRỌNG: LỌC NHÃN CHO RIÊNG CẶP 1-2 ---
    try:
        df_annot = pd.read_parquet(a_path)
        df_wide['label'] = 'other' # Mặc định là other
        
        # Chỉ lấy các hành động liên quan đến tương tác giữa Mouse 1 và Mouse 2
        # (Agent là 1, Target là 2) HOẶC (Agent là 2, Target là 1)
        pair_interaction = df_annot[
            ((df_annot['agent_id'] == 1) & (df_annot['target_id'] == 2)) |
            ((df_annot['agent_id'] == 2) & (df_annot['target_id'] == 1))
        ]
        
        for _, row in pair_interaction.iterrows():
            if row['stop_frame'] <= df_wide.index.max():
                df_wide.loc[row['start_frame']:row['stop_frame'], 'label'] = row['action']
                
    except:
        df_wide['label'] = 'unknown'

    return df_wide.fillna(0)

# --- CHẠY LẠI QUY TRÌNH TỪ ĐẦU (VỚI HÀM LỌC MỚI) ---
print("1. Load dữ liệu đã lọc (Chỉ cặp M1-M2)...")
df_train = get_video_data_filtered(0)
df_test = get_video_data_filtered(1)

if df_train is not None and df_test is not None:
    # Feature Engineering (Như cũ)
    print("2. Feature Engineering...")
    df_train = calculate_features_with_memory(df_train)
    df_test = calculate_features_with_memory(df_test)
    
    # Chuẩn bị Train/Test
    X_train = df_train[features]
    y_train_raw = df_train['label']
    X_test = df_test[features]
    y_test_raw = df_test['label']
    
    # Label Encoding
    le = LabelEncoder()
    le.fit(list(y_train_raw.unique()) + list(y_test_raw.unique()))
    y_train = le.transform(y_train_raw)
    y_test = le.transform(y_test_raw)
    
    # --- UNDERSAMPLING (Lại áp dụng bước 12) ---
    print("3. Cân bằng dữ liệu (Undersampling)...")
    df_train_processed = df_train.copy()
    df_train_processed['label_enc'] = y_train
    
    # Tách nhóm
    others = df_train_processed[df_train_processed['label_enc'] == le.transform(['other'])[0]]
    actions = df_train_processed[df_train_processed['label_enc'] != le.transform(['other'])[0]]
    
    # Cân bằng
    min_len = min(len(others), len(actions))
    # Nếu không có action nào (do lọc kỹ quá), ta phải handle
    if min_len > 0:
        others_down = resample(others, replace=False, n_samples=min_len, random_state=42)
        actions_down = resample(actions, replace=False, n_samples=min_len, random_state=42)
        df_final = pd.concat([others_down, actions_down])
        
        X_train_final = df_final[features]
        y_train_final = df_final['label_enc']
        
        # Train
        print(f"4. Train model (Dữ liệu sạch: {len(df_final)} dòng)...")
        model = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
        model.fit(X_train_final, y_train_final)
        
        # Test
        print("5. Dự đoán...")
        y_pred = model.predict(X_test)
        
        print(f"\n>>> ĐỘ CHÍNH XÁC: {accuracy_score(y_test, y_pred):.2%}")
        print(classification_report(y_test, y_pred, target_names=le.classes_, zero_division=0))
    else:
        print("Cảnh báo: Video này không có tương tác nào giữa Mouse 1 và Mouse 2! Hãy chọn video khác.")

1. Load dữ liệu đã lọc (Chỉ cặp M1-M2)...
Video 44566106: Đang xử lý cặp Mouse 1 & Mouse 2...
Video 143861384: Đang xử lý cặp Mouse 1 & Mouse 2...
2. Feature Engineering...
3. Cân bằng dữ liệu (Undersampling)...
4. Train model (Dữ liệu sạch: 1664 dòng)...
5. Dự đoán...

>>> ĐỘ CHÍNH XÁC: 88.09%
              precision    recall  f1-score   support

    approach       0.00      0.00      0.00         0
       avoid       0.00      0.11      0.00       126
       chase       0.00      0.00      0.00        97
       other       1.00      0.88      0.94     89442

    accuracy                           0.88     89665
   macro avg       0.25      0.25      0.24     89665
weighted avg       0.99      0.88      0.93     89665

